# The Three Body Problem  
Using Verlet intergration evolve a 3 body system for position and velocity.  
Calculate angular momentum of the bodies.  
Calculate total energy (potential and kinetic).  
### Verlet  
velocity half step  
$ v_{n+\frac{1}{2}} = v_n +\frac{1}{2} ha (x_n) $  
$ x_{n+1} = x_n +hv_{n+\frac{1}{2}} $  
$ v_{n+1} = v_{n+\frac{1}{2}} +\frac{1}{2} ha (x_{n+1}) $  

$ h $ is timestep  
$ a(x) $ is acceleration at point x. I will calulate using the force function defined near the top, and divide it by mass. So that it in keeping with the other solvers here.

In [1]:
#pip install numpy
#pip install matpoltlib
#pip install scipy
#pip install cycler
#pip install pandas
#pip install timeit
#pip install os

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import scipy.constants as sc
import pandas as pd
import timeit
import os

<jemalloc>: Unsupported system page size


### Functions

#### Data saving

In [3]:
def Data_save(p, v, E, L, t, f_name):
    """Function for saving positions, velocity, energy and angular momentum produced to a txt file.
    IN:
        p: An array holding all of the positions of that timestep. Cartesain
        v: An array holding all the velocities of that timestep. Cartesain
        E: Total energy of the system
        L: Total angular momentum of the system
        t: Time of the particular step
        f_name: The file the data is being saved to."""
    f = open(f_name, "a")
    f.write("\n")
    for i in range(len(p)):
        f.write(f"{p[i,0]}" + "\t" + f"{p[i,1]}" + "\t")
        
    for i in range (len(p)):
            f.write(f"{v[i,0]}" + "\t" + f"{v[i,1]}" + "\t")

    f.write(f"{E}" + "\t" + f"{L}" + "\t" +f"{t}")
    f.close()


#### Energies

In [4]:
def Grav_eng(pos_e, ma_e, pos_other, ma_other, ep, G):
    """Grav_eng(pos_e, ma_e, pos_other, ma_other):
    Calculates tthe gravtiational energy between two bodies.
    U = -GMm/(r+ep)
    IN:
        pos_e: position of the planet in question. [x,y]
        ma_e: mas of the planet in question.
        pos_other: position of the other planet. [x,y]
        ma_other: mas of the other planet.
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: GRavitational constant.
    OUT:
        g_e: gravitational potential energy."""
    
    dif = pos_other-pos_e
    g_e = - (G * ma_e *ma_other) / (np.linalg.norm(dif+ep))

    return g_e

In [5]:
def Kin_eng(vel_e, m_e):
    """Kin_eng(vel_e, m_e):
    Calculate the kinetic energy.
    E = 0.5mv^2
    IN:
        vel_e: velocity of a body. [vx,vy]
        m_e: mass of the planet in question.
    OUT:
        k_e: Kinetic energy of the body."""
    
    k_e = 0.5*m_e*vel_e**2
    k_e = np.linalg.norm(k_e)

    return k_e

#### Angular momentum

In [6]:
def Angular_momentum(p, v, m):
    """Angular_momentum(p, v, m):
    A function which calculates the angular momentum of the system.
    L = r x vm
    IN:
        p: position of the bodies.
        v: velocity of the bodies.
        m: mass of the bodies 
    OUT:
        a_m: total angular momentum for a time step."""

    rxv = np.cross(p,v)
    L = m * rxv
    
    return np.sum(L)

#### Force

In [7]:
def Force( ma_f, pos_f, ep, G):
    """Force( ma_f, pos_f, ep, G):
    A function for calculating a matricx of the gravitational force between bodies.
    F = GMm/r^2
    IN:
        ma_f: An array of all of the planetary masses.
        pos_f: position of the bodies. [x,y]
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: Gravitational constant.
    OUT:
        mat: a square matrix of the force between the planets"""
    size = len(ma_f)
    mat = np.zeros((size,size), dtype='object')
    mat[:] = 0
    for i in range(size):
        for j in range(size):
            if i==j:
                mat[i,j]=0
            else:
                mx = ma_f[i]
                my = ma_f[j]
                #dif = np.abs(pos[i]-pos[j])
                dif = (pos_f[i]-pos_f[j])
                force = - G*mx*my*dif / (np.linalg.norm(dif + ep)**3)
                #print(f" {i},{j} {force}")
                mat[i,j] = mat[i,j]+force
                
    #print(f"force matrix {mat}")
    
    mat = np.sum(mat, axis=1)
    #print(f" force {mat}")
    return mat

#### Verlet step

In [8]:
def Verlet(pos_v, vel_v, ma, ep, G, t_s):
    """A function that runs a single loop of a verlet integrator.
    IN:
        pos_v: an array holding the positions of all the planets at the beginning of a time step.
        vel_v: an array holding the velocities at the beginning of a time step.
        ma: an array holding the mass of all of the planets.
        ep: extra bit to make sure the numbers dont blow up too much at low distances.
        G: gravitational constant.
        t_s: time step.

    OUT:
        pos_full: an array holding the position of all the bodies at the end of a time step
        vel_full: an array holding the velocities of all the bodies at the end of a time step
    """

    pos_full = np.array([])
    vel_full = np.array([])
    vel_half = np.array([])
    kinetic = np.array([])
    gravitational = np.array([])
    # Calculating halfstep forces
    f_half = Force(ma_f=ma, pos_f=pos_v, ep=ep, G=G)

    # Calculate half-step velocities and full-step positions
    for v in range(len(ma)):
        v_h = vel_v[v] + (0.5 * t_s * f_half[v]/ma[v])
        p_full = pos_v[v] + (t_s * v_h)
        
        # Store half-step velocity
        if vel_half.shape == (0,):
            vel_half = np.array([v_h])
        else:
            vel_half = np.vstack((vel_half, np.array([v_h])))
            
        # Store full-step position
        if pos_full.shape == (0,):
            pos_full = np.array([p_full])
        else:
            pos_full = np.vstack((pos_full, np.array([p_full])))
    
    # Calculating full step forces
    f_full = Force(ma_f=ma, pos_f=pos_full, ep=ep, G=G)

    # Calculate full-step velociteis using stored half-step velocities
    for ve in range(len(ma)):
        v_full = vel_half[ve] + (0.5 * t_s * f_full[ve]/ma[ve])

        # Storing full step velocities
        if vel_full.shape == (0,):
            vel_full = np.array([v_full])
        else:
            vel_full = np.vstack((vel_full, np.array([v_full])))
        # Calcaulting energies
        kinetic = np.append(kinetic, Kin_eng(v_full, ma[ve]))
        gravitational = np.append(gravitational, Grav_eng(pos_full[ve], ma[ve], pos_full, ma, ep, G))

    kinetic = np.sum(kinetic)
    gravitational = np.sum(gravitational)
    return pos_full, vel_full, gravitational, kinetic

### Main

#### Simulating

In [9]:
def Running_simulations(da, si, na):
    """Function which runs the verlet nbody simulations.
    In:
        da: Input data.
        si: Simulations to run.
        na: names fo the simualtions"""
    
    print(f"number of simulations running {len(simulations)}")
    for s in range(len(simulations)):

        #Selecting input data
        r = si[s]
        in_con = da.iloc[r]
        name = na[r]

        
        masses = np.array([in_con.loc["m1"], in_con.loc["m2"], in_con.loc["m3"]])
        i_positions = np.array([[in_con.loc["pos1x"],in_con.loc["pos1y"]], [in_con.loc["pos2x"],in_con.loc["pos2y"]], [in_con.loc["pos3x"],in_con.loc["pos3y"]]])
        i_velocities = np.array([[in_con.loc["vel1x"],in_con.loc["vel1y"]], [in_con.loc["vel2x"],in_con.loc["vel2y"]], [in_con.loc["vel3x"],in_con.loc["vel3y"]]])
        
        
        fin = in_con.loc["period"] * 1.5
        time_step = in_con.loc["tstep"]
        no_steps = round(fin/time_step)
        
        print(f"Simulation {name}, system {r}, {no_steps} steps")
        
        epsilon = in_con.loc["epsilon"]
        stated_energy = in_con.loc["energy"]
        stated_angmom = in_con.loc["angmom"]
        G = 1

        for i in range(len(masses)):
            if masses[i] == 0:
                masses = np.delete(masses, i)
                i_positions = np.delete(i_positions, i, 0)
                i_velocities = np.delete(i_velocities, i, 0)

        # making a folder to hold produced data
        if os.path.exists("n_body_data") == False:
            os.makedirs("n_body_data")
            print("folder 'n_body_data' created")

        # Making a data file
        file_name = f"n_body_data/produced_data_system_{name}.txt"
        file = open(file_name, "w")
    
        for a in range(len(masses)):
            file.write(f"p{a+1}x" + "\t" + f"p{a+1}y" + "\t")
        
        for a in range(len(masses)):
            file.write(f"v{a+1}x" + "\t" + f"v{a+1}y" + "\t")
    
        file.write("Energy" + "\t" + "Ang_mom" + "\t" + "time")
        
        file.close()
    
        pos = i_positions
        vel = i_velocities
    
    
        tot_eng=stated_energy
        ang_mom=stated_angmom
        
        t1 = timeit.default_timer()

        # Running the Verlet intergrator
        for i in range(no_steps):
    
            time = i *time_step
        
            Data_save(p=pos, v=vel, E=tot_eng, L=ang_mom, t=time, f_name = file_name)
        
            new_pos, new_vel, gravitational_energy, kinetic_energy = Verlet(pos_v=pos, vel_v=vel, ma=masses, ep=epsilon, G=G, t_s=time_step)     #Calling verlet intergrator
        
            ang_mom = Angular_momentum(p = new_pos, v = new_vel, m = masses)
            tot_eng = kinetic_energy + np.sum(gravitational_energy)
    
            pos = new_pos
            vel = new_vel
        
    
        t2 = timeit.default_timer()
        print(f"system {r} time taken {t2-t1}")

#### Plotting

In [10]:
def Plotting_simualtions(pl,na):
    """Function which plot the data produced by the simulations.
    In:
        pl: simulations to be plotted.
        na: Names of simulations to be plotted."""
    for p in range(len(pl)):
        t1 = timeit.default_timer()

        #Slecting the data to plot
        r = pl[p]
        name = na[r]
        produced_data_name = f"n_body_data/produced_data_system_{name}.txt"
        produced_data = pd.read_csv(produced_data_name,sep='\s+')
        print(f"Plot {name}, system{r}")
        
        #producing the plots
        fig =  plt.figure()
        ax = fig.add_subplot()
        fig_E_L, ax_E_L = plt.subplots(2,3, figsize = (10,8))
        #print(produced_data)
        #print(produced_data.columns)
        #print(produced_data["p1x"])
        no_masses = int((produced_data.shape[1] -3)/4)     # calculating the number of masses in the simulation by number of columns removing the number for energy, ang_mom and time. Leaving just position and velocity then quarter of the remaining columns are no masses
        col = cm.rainbow(np.linspace(0, 1, no_masses))     #colour map to make each body a different colour

        #Plotting the bodies
        for no in range(no_masses):
            #alp = np.linspace(0, 1, len(produced_data[f"p{no+1}x"]))
            alp = np.logspace(0.0, 1.0, len(produced_data[f"p{no+1}x"])) / 10.
            ax.scatter(produced_data[f"p{no+1}x"], produced_data[f"p{no+1}y"], color = col[no], alpha = alp, s=0.5)

        ax.grid(True)
        
        ti = np.delete(produced_data["time"],0)
        en= np.delete(produced_data["Energy"],0)
        an = np.delete(produced_data["Ang_mom"],0)

        
        #difference with the previous
        en_change = np.diff(en, n=1) * 100
        an_change = np.diff(an, n=1) * 100

        
        en_claimed = produced_data["Energy"][0]
        an_claimed = produced_data["Ang_mom"][0]

        
        #difference with the stated
        en_dif =  100 * np.abs(en-en_claimed)/(2 *(en+en_claimed))
        an_dif =  100 * np.abs(an-an_claimed)/(2 *(an+an_claimed))


        
        #Plotting the energy and angular momentum
        ax_E_L[0, 0].scatter(ti, en, color = "Green", s=1)
        ax_E_L[1, 0].scatter(ti, an, color = "Blue", s=1)
        ax_E_L[0, 1].scatter(np.delete(ti,0), en_change, color = "Green", s=1) 
        ax_E_L[1, 1].scatter(np.delete(ti,0), an_change, color = "Blue", s=1)
        ax_E_L[0, 2].scatter(ti, en_dif, color = "Green", s=1) 
        ax_E_L[1, 2].scatter(ti, an_dif, color = "Blue", s=1)

        fig_E_L.suptitle(f"{name}", fontsize="large")
        ax_E_L[0,0].set_title("Total System Energy")
        ax_E_L[1,0].set_title("Total System Ang mom")
        ax_E_L[0,1].set_title("Energy Change")
        ax_E_L[1,1].set_title("Ang mom Change")
        ax_E_L[0,2].set_title("Energy Diffference")
        ax_E_L[1,2].set_title("Ang mom Diffference")

        ax_E_L[0,0].set_xlabel("t / s")
        ax_E_L[0,0].set_ylabel("E / J")
        ax_E_L[1,0].set_xlabel("t / s")
        ax_E_L[1,0].set_ylabel("L / kg m^2 s^-1")
        ax_E_L[0,1].set_xlabel("t / s")
        ax_E_L[0,1].set_ylabel("%")
        ax_E_L[1,1].set_xlabel("t / s")
        ax_E_L[1,1].set_ylabel("%")
        ax_E_L[0,2].set_xlabel("t / s")
        ax_E_L[0,2].set_ylabel("%")
        ax_E_L[1,2].set_xlabel("t / s")
        ax_E_L[1,2].set_ylabel("%")
        
        fig.tight_layout()
        fig_E_L.tight_layout()
        confine = 2.5
        #ax.set_xlim(-confine,confine)
        #ax.set_ylim(-confine,confine)

        #Making a folder for the plots and saving them to it
        if os.path.exists("n_body_plots") == False:
            os.makedirs("n_body_plots")
            print("folder 'n_body_plots' created")
            
        fig.savefig(f"n_body_plots/system_{name}_nbody_sim.png", dpi=250)
        fig_E_L.savefig(f"n_body_plots/system_{name}_energy_angmom.png", dpi=250)
        
        plt.close(fig)
        plt.close(fig_E_L)
        t2 = timeit.default_timer()
        print(f"system {r} time taken {t2-t1}")

In [11]:
#data = pd.read_csv("3_body_input_and_tests_2.txt",sep='\s+')
data = pd.read_csv("3_body_input_and_tests_0.txt",sep='\s+')
names = data.iloc[:,0]

In [12]:
data

,name,pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,vel1x,vel1y,vel2x,...,vel3x,vel3y,m1,m2,m3,tstep,epsilon,period,energy,angmom
0,Test1,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,100,1,0,0.00010,0.000000,3.000000,0.000000,0
1,Test2,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,100000,1,0,0.00001,0.000000,3.000000,0.000000,0
2,Test3,0.000000,0.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,100000,1,0,0.00001,0.000000,3.000000,0.000000,0
3,Test4,1.000000,1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1000,1000,0,0.00010,0.000000,1.000000,0.000000,0
4,Test5,1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,10,10,0,0.00010,0.000000,3.000000,0.000000,0
5,Test6,1.000000,3.000000,2.000000,2.000000,0.000000,0.000000,3.000000,2.000000,1.000000,...,0.000000,0.000000,2,2,0,0.00010,0.000000,10.000000,0.000000,0
6,Test7,1.000000,1.000000,-1.000000,-2.000000,3.000000,-2.000000,1.000000,1.000000,2.000000,...,-1.000000,-2.000000,1,1,1,0.00010,0.000000,10.000000,0.000000,0
7,Test8,0.000000,0.000000,2.000000,0.000000,1.000000,1.732000,0.000000,0.000000,0.000000,...,0.000000,0.000000,5,5,5,0.00010,0.000000,2.000000,0.000000,0
8,Test9,1.000000,1.000000,-2.000000,1.000000,4.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,-5.000000,100,5,5,0.00010,0.000000,10.000000,0.000000,0
9,Test10,1.000000,1.000000,-2.000000,1.000000,4.000000,1.000000,1.000000,-0.500000,0.000000,...,0.000000,-5.000000,100,5,5,0.00010,0.000000,10.000000,0.000000,0


### Running

In [13]:
simulating = True    #true to simulate,false to not
simulations = np.array([20])
#simulations = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20])

plotting = True     #true to plot,false to not
#plots = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
plots = simulations

In [14]:
if simulating ==True:
    Running_simulations(da=data, si=simulations, na=names)

if plotting == True:
    Plotting_simualtions(pl = plots, na=names)

number of simulations running 1
Simulation pt2, system 20, 814198 steps


/tmp/ipykernel_48379/1474752069.py:12: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  rxv = np.cross(p,v)


system 20 time taken 385.5629949110007
Plot pt2, system20
system 20 time taken 80.88394301399967


In [15]:
print("finished")

finished
